#### guide:
https://python.langchain.com/v0.2/docs/how_to/custom_tools/


# Creating tools from functions
### @tool decorator
This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

In [1]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.


In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
multiply({'a':3,'b':5})

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


15

In [3]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


# Let's inspect some of the attributes associated with the tool.


In [4]:
@tool
def divide(a: int, b: int) -> int:
    """Perform division of a number"""
    return a/b


# Let's inspect some of the attributes associated with the tool.


In [5]:
@tool
def subtract(a: int, b: int) -> int:
    """Perform Subtraction"""
    return a-b


# Let's inspect some of the attributes associated with the tool.


In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [add, multiply,subtract,divide]
llm_with_tools = llm.bind_tools(tools)

In [7]:
llm_with_tools.invoke("What is 98/9")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YU2hWFSWcRFaauKycQWFLlSg', 'function': {'arguments': '{"a":98,"b":9}', 'name': 'divide'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 115, 'total_tokens': 132}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ac864afa-c5c3-4665-9107-3c48fb485bc4-0', tool_calls=[{'name': 'divide', 'args': {'a': 98, 'b': 9}, 'id': 'call_YU2hWFSWcRFaauKycQWFLlSg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 115, 'output_tokens': 17, 'total_tokens': 132})

In [8]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [10]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({"input": "What is 8 divided by 2!"})



> Entering new AgentExecutor chain...

Invoking: `divide` with `{'a': 8, 'b': 2}`


4.0The result of \( 8 \div 2! \) is 4.0.

> Finished chain.


{'input': 'What is 8 divided by 2!',
 'output': 'The result of \\( 8 \\div 2! \\) is 4.0.'}

In [12]:
agent_executor.invoke({"input": "What is 88*6!"})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 88, 'b': 720}`


63360The value of \( 88 \times 6! \) is 63,360.

> Finished chain.


{'input': 'What is 88*6!',
 'output': 'The value of \\( 88 \\times 6! \\) is 63,360.'}

In [13]:
agent_executor.invoke({"input": "if i have 3 apples and i get 567 more apples. How many apples will I have in all?"})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 3, 'b': 567}`


570You will have a total of 570 apples.

> Finished chain.


{'input': 'if i have 3 apples and i get 567 more apples. How many apples will I have in all?',
 'output': 'You will have a total of 570 apples.'}

In [14]:
agent_executor.invoke({"input": "Who is Elon Musk?"})



> Entering new AgentExecutor chain...
Elon Musk is a billionaire entrepreneur and business magnate known for his role in various high-profile technology companies. Born on June 28, 1971, in Pretoria, South Africa, he moved to the United States to pursue his education and career. Musk is the CEO and lead designer of SpaceX, a private aerospace manufacturer and space transportation company, and the CEO and product architect of Tesla, Inc., which specializes in electric vehicles and renewable energy solutions.

In addition to these roles, Musk has been involved in several other ventures:

1. **PayPal**: He co-founded X.com, an online payment company that later became PayPal after a merger.
2. **Neuralink**: A neurotechnology company focused on developing brain-computer interfaces.
3. **The Boring Company**: A tunnel construction and infrastructure company.
4. **SolarCity**: A solar energy services company that was acquired by Tesla.

Musk is known for his vision of the future, which inc

{'input': 'Who is Elon Musk?',
 'output': 'Elon Musk is a billionaire entrepreneur and business magnate known for his role in various high-profile technology companies. Born on June 28, 1971, in Pretoria, South Africa, he moved to the United States to pursue his education and career. Musk is the CEO and lead designer of SpaceX, a private aerospace manufacturer and space transportation company, and the CEO and product architect of Tesla, Inc., which specializes in electric vehicles and renewable energy solutions.\n\nIn addition to these roles, Musk has been involved in several other ventures:\n\n1. **PayPal**: He co-founded X.com, an online payment company that later became PayPal after a merger.\n2. **Neuralink**: A neurotechnology company focused on developing brain-computer interfaces.\n3. **The Boring Company**: A tunnel construction and infrastructure company.\n4. **SolarCity**: A solar energy services company that was acquired by Tesla.\n\nMusk is known for his vision of the futur